<a href="https://colab.research.google.com/github/hrcosko/master_th_DL_segm/blob/main/Diplosmki_Coskovic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Crash on purpose to get more ram :
import torch
torch.tensor([10.]*10000000000)

___ 
# data
___

In [ ]:
import os
from datetime import datetime
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
%load_ext tensorboard

In [ ]:
def load_data(path, split=0.1):
    
    images = sorted(glob(os.path.join(path, 'MP6843_img/*')))
    masks = glob(os.path.join(path, 'MP6843_mask/*'))
    masks = masks + masks
    masks.sort()

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


In [ ]:
def read_image(path):
    
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (0,0), fx=0.5, fy=0.5)
    image = image/255.0
    
    return image

In [ ]:
def read_mask(path):
    
    path = path.decode()
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = mask/255.0
    mask = np.expand_dims(mask, axis=-1)
    
    return mask

In [ ]:
def tf_dataset(image, mask, batch=4):

    def parse(image, mask):
        image = read_image(image)
        mask = read_mask(mask)
        return image, mask

    def tf_parse(image, mask):
      image, mask = tf.numpy_function(parse, [image, mask], [tf.float64, tf.float64])
      image.set_shape([520, 696, 3])
      mask.set_shape([520, 696, 1])
      return image, mask
    
    def data_augment_flip_rl(image, mask):
      image, mask = tf.numpy_function(parse, [image, mask], [tf.float64, tf.float64])
      modified_image = tf.image.flip_left_right(image)
      modified_mask = tf.image.flip_left_right(mask)
      modified_image.set_shape([520, 696, 3])
      modified_mask.set_shape([520, 696, 1])
      return modified_image, modified_mask

    def data_augment_flip_up_down(image, mask):
      image, mask = tf.numpy_function(parse, [image, mask], [tf.float64, tf.float64])
      modified_image = tf.image.flip_up_down(image)
      modified_mask = tf.image.flip_up_down(mask)
      modified_image.set_shape([520, 696, 3])
      modified_mask.set_shape([520, 696, 1])
      return modified_image, modified_mask
    
    #def data_augment_grayscale(image, mask):
    #  image, mask = tf.numpy_function(parse, [image, mask], [tf.float64, tf.float64])
    #  modified_image = tf.image.rgb_to_grayscale(image)
    #  modified_image.set_shape([520, 696, 3])
    #  mask.set_shape([520, 696, 1])
    #  return modified_image, mask
  
    def data_augment_saturnate(image, mask):
      image, mask = tf.numpy_function(parse, [image, mask], [tf.float64, tf.float64])
      modified_image = tf.image.adjust_saturation(image, 3)
      modified_image.set_shape([520, 696, 3])
      mask.set_shape([520, 696, 1])
      return modified_image, mask
    
    def data_augment_rot(image, mask):
      image, mask = tf.numpy_function(parse, [image, mask], [tf.float64, tf.float64])
      modified_image = tf.image.rot90(image)
      modified_mask = tf.image.rot90(mask)
      modified_image = tf.image.rot90(modified_image)
      modified_mask = tf.image.rot90(modified_mask)
      modified_image.set_shape([520, 696, 3])
      modified_mask.set_shape([520, 696, 1])
      return modified_image, modified_mask

    dataset = tf.data.Dataset.from_tensor_slices((image, mask))
    dataset1 = tf.data.Dataset.from_tensor_slices((image, mask))
    dataset2 = tf.data.Dataset.from_tensor_slices((image, mask))
    dataset3 = tf.data.Dataset.from_tensor_slices((image, mask))
    dataset4 = tf.data.Dataset.from_tensor_slices((image, mask))
    dataset5 = tf.data.Dataset.from_tensor_slices((image, mask))  
    
    dataset = dataset.map(tf_parse)
    dataset1 = dataset1.map(data_augment_flip_rl)
    dataset2 = dataset2.map(data_augment_flip_up_down)
    #dataset3 = dataset3.map(data_augment_grayscale)
    dataset4 = dataset4.map(data_augment_rot)
    dataset5 = dataset5.map(data_augment_saturnate)
    
    print('DS0: ', dataset.cardinality().numpy(), ' ###### ', dataset)
    dataset = dataset.concatenate(dataset1)
    print('DS1: ', dataset.cardinality().numpy(), ' ###### ', dataset1)
    dataset = dataset.concatenate(dataset2)
    print('DS2: ', dataset.cardinality().numpy(), ' ###### ', dataset2)
    #dataset = dataset.concatenate(dataset3)
    #print('DS3: ', dataset.cardinality().numpy(), ' ###### ', dataset3)
    dataset = dataset.concatenate(dataset4)
    print('DS4: ', dataset.cardinality().numpy(), ' ###### ', dataset4)
    dataset = dataset.concatenate(dataset5)
    print('DS5: ', dataset.cardinality().numpy(), ' ###### ', dataset5)
    dataset.shuffle(2000)
    print('DSshuffle: ', dataset.cardinality().numpy(), ' ###### ', dataset)
    dataset = dataset.batch(batch)
    print('DSbatch: ', dataset.cardinality().numpy(), ' ###### ', dataset)
    #dataset = dataset.repeat()
    
    return dataset

In [ ]:
path = "/content/drive/MyDrive/DIPLOMSKI/code/dataset/"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

In [ ]:
#for image, mask in train_dataset.take(1):
#  print("Image shape: ", image.numpy().shape)
#  image = image.numpy()
#  image = np.reshape(image, (-1, 520, 696, 3))
#  print("Label: ", mask.numpy().shape)

In [ ]:
logdir = '/content/drive/MyDrive/DIPLOMSKI/code/logs/'
!rm -rf '/content/drive/MyDrive/DIPLOMSKI/code/logs/'

In [ ]:
#logdir += 'train_data/' + datetime.now().strftime("%Y%m%d-%H%M%S")
#file_writer = tf.summary.create_file_writer(logdir)
#
#with file_writer.as_default():
#  tf.summary.image("Training data", image, step=0)

In [ ]:
# %tensorboard --logdir /content/drive/MyDrive/DIPLOMSKI/code/logs/train_data

___
# model
___

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from types import MethodType
import keras.backend as K

In [ ]:
def build_model(n_classes=1, input_height=520, input_width=696, 
                kernel=3, dropout=0.3, pool_size = 2):

    ## BlueEncoder
    filter_size = 64
    img_input = Input(shape=(input_height, input_width, 3))

    x = img_input
    levels = []
    
    
    x = Conv2D(filter_size, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    block_input = x
    x = Dropout(dropout)(x)
    x = Conv2D(filter_size, (kernel, kernel), padding='same')(x)
    x = Add()([x, block_input])
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    levels.append(x)
    
    x = MaxPooling2D((pool_size, pool_size))(x)
    
    x = Conv2D(128, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    block_input = x
    x = Dropout(dropout)(x)
    x = Conv2D(128, (kernel, kernel), padding='same')(x)
    x = Add()([x, block_input])
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    levels.append(x)
    
    x = MaxPooling2D((pool_size, pool_size))(x)
    
    x = Conv2D(256, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    block_input = x
    x = Dropout(dropout)(x)
    x = Conv2D(256, (kernel, kernel), padding='same')(x)
    x = Add()([x, block_input])
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    levels.append(x)
    
    x = MaxPooling2D((pool_size, pool_size))(x)
   
    x = Conv2D(512, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    block_input = x
    x = Dropout(dropout)(x)
    x = Conv2D(512, (kernel, kernel), padding='same')(x)
    x = Add()([x, block_input])
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    aux_f4 = x
    x = ZeroPadding2D(((0, 1), (0, 1)))(x)
    levels.append(x)
    
    x = MaxPooling2D((pool_size, pool_size))(x)
    
    x = Conv2D(1024, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    block_input = x
    x = Dropout(dropout)(x)
    x = Conv2D(1024, (kernel, kernel), padding='same')(x)
    x = Add()([x, block_input])
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    levels.append(x)
    
    [f1, f2, f3, f4, f5] = levels
    #for _ in levels:
      #print(_)

    ### MidYellowDecoder

    m1 = Conv2DTranspose(64, kernel_size=(2, 2), strides=(2,2))(f2)
    m1 = concatenate([f1, m1], axis=-1)
    m1 = Conv2D(64, (3, 3), padding='same')(m1)
    m1 = BatchNormalization()(m1)
    m1 = Activation('relu')(m1)
    m1 = Conv2D(64, (3, 3), padding='same')(m1)
    m1 = BatchNormalization()(m1)
    m1 = Activation('relu')(m1)
    m1 = Dropout(dropout)(m1)

    m2 = Conv2DTranspose(128, kernel_size=(2, 2), strides=(2,2))(f3)
    m2 = concatenate([f2, m2], axis=-1)
    m2 = Conv2D(128, (3, 3), padding='same')(m2)
    m2 = BatchNormalization()(m2)
    m2 = Activation('relu')(m2)
    m2 = Conv2D(128, (3, 3), padding='same')(m2)
    m2 = BatchNormalization()(m2)
    m2 = Activation('relu')(m2)
    m2 = Dropout(dropout)(m2)

    m3 = Conv2DTranspose(256, kernel_size=(2, 2), strides=(2,2))(aux_f4)
    m3 = concatenate([f3, m3], axis=-1)
    m3 = Conv2D(256, (3, 3), padding='same')(m3)
    m3 = BatchNormalization()(m3)
    m3 = Activation('relu')(m3)
    m3 = Conv2D(256, (3, 3), padding='same')(m3)
    m3 = BatchNormalization()(m3)
    m3 = Activation('relu')(m3)
    m3 = Dropout(dropout)(m3)

    ### RedNumaDecoder

    n1 = Conv2DTranspose(64, kernel_size=(2, 2), strides=(2,2))(m2)
    n1 = concatenate([f1, m1, n1], axis=-1)
    n1 = Conv2D(64, (3, 3), padding='same')(n1)
    n1 = BatchNormalization()(n1)
    n1 = Activation('relu')(n1)
    n1 = Conv2D(64, (3, 3), padding='same')(n1)
    n1 = BatchNormalization()(n1)
    n1 = Activation('relu')(n1)
    n1 = Dropout(dropout)(n1)

    n2 = Conv2DTranspose(128, kernel_size=(2, 2), strides=(2,2))(m3)
    n2 = concatenate([f2, m2, n2], axis=-1)
    n2 = Conv2D(128, (3, 3), padding='same')(n2)
    n2 = BatchNormalization()(n2)
    n2 = Activation('relu')(n2)
    n2 = Conv2D(128, (3, 3), padding='same')(n2)
    n2 = BatchNormalization()(n2)
    n2 = Activation('relu')(n2)
    n2 = Dropout(dropout)(n2)

    ### LonleyDecoder

    l1 = Conv2DTranspose(64, kernel_size=(2, 2), strides=(2,2))(n2)
    l1 = concatenate([f1, m1, n1, l1], axis=-1)
    l1 = Conv2D(64, (3, 3), padding='same')(l1)
    l1 = BatchNormalization()(n1)
    l1 = Activation('relu')(l1)
    l1 = Conv2D(64, (3, 3), padding='same')(l1)
    l1 = BatchNormalization()(l1)
    l1 = Activation('relu')(l1)
    l1 = Dropout(dropout)(l1)


    ### GreenOutputDecoder
    o = f5
    
    o = Conv2DTranspose(512, kernel_size=(2, 2), strides=(2,2))(o)
    
    o = concatenate([o, f4], axis=-1)
    o = Conv2D(512, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)
    o = Dropout(dropout)(o)
    o = Conv2D(512, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)
    o = Cropping2D(((0, 1), (0, 1)))(o)
    
    o = Conv2DTranspose(256, kernel_size=(2, 2), strides=(2,2))(o)
        
    o = concatenate([o, f3, m3], axis=-1)
    o = Conv2D(256, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)
    o = Dropout(dropout)(o)
    o = Conv2D(256, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)

    o = Conv2DTranspose(128, kernel_size=(2, 2), strides=(2,2))(o)

    o = concatenate([o, f2, m2, n2], axis=-1)
    o = Conv2D(128, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)
    o = Dropout(dropout)(o)
    o = Conv2D(128, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)

    o = Conv2DTranspose(64, kernel_size=(2, 2), strides=(2,2))(o)

    o = concatenate([o, f1, m1, n1, l1], axis=-1)
    o = Conv2D(64, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)
    o = Dropout(dropout)(o)
    o = Conv2D(64, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)

    ### Output
    o = Conv2D(n_classes, (1, 1), padding='same')(o)
    o = Activation('sigmoid')(o)
    model = Model(img_input, o)
    
    return model
    

In [ ]:
model = build_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 520, 696, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 520, 696, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 520, 696, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 520, 696, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
#dot_img_file = '/content/drive/MyDrive/DIPLOMSKI/code/model_1.png'
#tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

---
# train
---



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard

In [ ]:
def iou(y_true, y_pred):
     
     def f(y_true, y_pred):
         
         intersection = (y_true * y_pred).sum()
         union = y_true.sum() + y_pred.sum() - intersection
         x = (intersection + 1e-15) / (union + 1e-15)
         x = x.astype(np.float32)
         
         return x
     
     return tf.numpy_function(f, [y_true, y_pred], tf.float32)

def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1. - dice_coef(y_true, y_pred)

In [ ]:
lr = 1e-4 * 3.1415 
epochs = 20
batch = 2

In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch)

DS0:  160  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DS1:  320  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DS2:  480  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DS4:  640  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DS5:  800  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DSshuffle:  800  ######  <ConcatenateDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DSbatch:  400  ######  <BatchDataset shapes: ((None, 520, 696, 3), (None, 520, 696, 1)), types: (tf.float64, tf.float64)>
DS0:  20  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DS1:  40  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DS2:  60  ######  <MapDataset shap

In [ ]:
model = build_model()

opt = tf.keras.optimizers.Adam(lr)
metrics = [dice_coef, tf.keras.metrics.Recall(), iou]
model.compile(loss=dice_coef_loss, optimizer=opt, metrics=metrics)

In [ ]:
callbacks = [
          ModelCheckpoint("/content/drive/MyDrive/DIPLOMSKI/code/files/model_new.h5"),
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
          CSVLogger("/content/drive/MyDrive/DIPLOMSKI/code/files/data.csv"),
          TensorBoard()
     ]

In [ ]:
train_steps = train_dataset.cardinality().numpy() 
valid_steps = valid_dataset.cardinality().numpy() 

if train_steps % batch != 0:
    train_steps += 1

if valid_steps % batch != 0:
    valid_steps += 1

In [ ]:
train_dataset = train_dataset.repeat()
valid_dataset = valid_dataset.repeat()

In [ ]:
model.fit(train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks)

model.save("/content/drive/MyDrive/DIPLOMSKI/code/files/saved_model.h5")

Epoch 1/20


In [ ]:
model.save("/content/drive/MyDrive/DIPLOMSKI/code/files/saved_model.h5")

___
#predict
___

In [ ]:
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm

In [ ]:
#reloaded_model = tf.keras.models.load_model('/content/drive/MyDrive/DIPLOMSKI/code/model.h5')

In [ ]:
def mask_parse(mask):
     mask = np.squeeze(mask)
     mask = [mask, mask]
     mask = np.transpose(mask, (1, 2, 0))
     return mask

def read_image(path):
     x = cv2.imread(path, cv2.IMREAD_COLOR)
     x = cv2.resize(x, (0,0), fx=0.5, fy=0.5)
     x = x/255.0
     return x

def read_mask(path):
     x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
     #x = cv2.resize(x, (256, 256))
     x = np.expand_dims(x, axis=-1)
     return x

In [ ]:
for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
  x = read_image(x)
  y = read_mask(y)
  y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
  y_pred = y_pred.astype(np.uint8) * 255.0
  print(y_pred)
  cv2.imwrite(f"/content/drive/MyDrive/DIPLOMSKI/code/results/{i}_GT_2021_02_07.png", y)
  cv2.imwrite(f"/content/drive/MyDrive/DIPLOMSKI/code/results/{i}_PRED_2021_02_07.png", y_pred)

100%|██████████| 20/20 [00:44<00:00,  2.20s/it]


In [ ]:
#reloaded_model = tf.keras.models.load_model('model.h5')

with CustomObjectScope({'iou': iou}):
  reloaded_model = tf.keras.models.load_model('/content/drive/MyDrive/DIPLOMSKI/code/files/2021_01_29/saved_model.h5')

In [ ]:
reloaded_model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
reloaded_model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)

In [ ]:
reloaded_model.fit(train_dataset,
    validation_data=valid_dataset,
    epochs=50,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks)


Epoch 1/50
200/200 [==============================] - 930s 4s/step - loss: 0.1713 - acc: 0.9300 - recall: 0.8018 - precision: 0.8625 - iou: 0.6143 - val_loss: 0.4120 - val_acc: 0.8449 - val_recall: 0.3492 - val_precision: 0.8885 - val_iou: 0.2810
Epoch 2/50
200/200 [==============================] - 385s 2s/step - loss: 0.1660 - acc: 0.9317 - recall: 0.8047 - precision: 0.8676 - iou: 0.6218 - val_loss: 0.3247 - val_acc: 0.8915 - val_recall: 0.7040 - val_precision: 0.7876 - val_iou: 0.3887
Epoch 3/50
200/200 [==============================] - 383s 2s/step - loss: 0.1641 - acc: 0.9322 - recall: 0.8052 - precision: 0.8693 - iou: 0.6251 - val_loss: 0.3629 - val_acc: 0.8613 - val_recall: 0.8807 - val_precision: 0.6371 - val_iou: 0.4151
Epoch 4/50
200/200 [==============================] - 383s 2s/step - loss: 0.1604 - acc: 0.9335 - recall: 0.8090 - precision: 0.8719 - iou: 0.6322 - val_loss: 0.4363 - val_acc: 0.8394 - val_recall: 0.3212 - val_precision: 0.8887 - val_iou: 0.2968
Epoch 5/50
2

KeyboardInterrupt: ignored

In [ ]:
reloaded_model.save("/content/drive/MyDrive/DIPLOMSKI/code/files/saved_model.h5")

NameError: ignored

# evaluate

In [ ]:
evaluate_dataset = tf_dataset(test_x, test_y, batch=1)

DS0:  20  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DS1:  40  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DS2:  60  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DS4:  80  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DS5:  100  ######  <MapDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DSshuffle:  100  ######  <ConcatenateDataset shapes: ((520, 696, 3), (520, 696, 1)), types: (tf.float64, tf.float64)>
DSbatch:  100  ######  <BatchDataset shapes: ((None, 520, 696, 3), (None, 520, 696, 1)), types: (tf.float64, tf.float64)>


In [ ]:
with CustomObjectScope({'iou': iou}):
  reloaded_model = tf.keras.models.load_model('/content/drive/MyDrive/DIPLOMSKI/code/files/2021_02_06/model_new.h5')

In [ ]:
opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou, dice_coef]
reloaded_model.compile(loss=["binary_crossentropy", dice_coef_loss], optimizer=opt, metrics=metrics)

In [ ]:
reloaded_model.fit(train_dataset,
    validation_data=valid_dataset,
    epochs=10,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks)

Epoch 1/10
800/800 [==============================] - 1206s 1s/step - loss: 0.1566 - acc: 0.9347 - recall_1: 0.8095 - precision_1: 0.8773 - iou: 0.6525 - dice_coef: 0.7783 - val_loss: 0.4438 - val_acc: 0.8395 - val_recall_1: 0.3101 - val_precision_1: 0.8714 - val_iou: 0.2885 - val_dice_coef: 0.4277
Epoch 2/10
800/800 [==============================] - 827s 1s/step - loss: 0.1574 - acc: 0.9344 - recall_1: 0.8059 - precision_1: 0.8791 - iou: 0.6517 - dice_coef: 0.7775 - val_loss: 0.3592 - val_acc: 0.8694 - val_recall_1: 0.6385 - val_precision_1: 0.7293 - val_iou: 0.3536 - val_dice_coef: 0.5148
Epoch 3/10
800/800 [==============================] - 828s 1s/step - loss: 0.1527 - acc: 0.9363 - recall_1: 0.8109 - precision_1: 0.8830 - iou: 0.6588 - dice_coef: 0.7827 - val_loss: 0.5229 - val_acc: 0.8291 - val_recall_1: 0.3436 - val_precision_1: 0.7303 - val_iou: 0.2781 - val_dice_coef: 0.4164
Epoch 4/10
800/800 [==============================] - 826s 1s/step - loss: 0.1499 - acc: 0.9372 - reca

In [ ]:
for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
  x = read_image(x)
  y = read_mask(y)
  y_pred = reloaded_model.predict(np.expand_dims(x, axis=0))[0] > 0.5
  y_pred = y_pred.astype(np.uint8) * 255.0
  cv2.imwrite(f"/content/drive/MyDrive/DIPLOMSKI/code/results/{i}_GT_2021_02_07.png", y)
  cv2.imwrite(f"/content/drive/MyDrive/DIPLOMSKI/code/results/{i}_PRED_2021_02_07.png", y_pred)

  0%|          | 0/20 [00:00<?, ?it/s]


TypeError: ignored

In [ ]:
reloaded_model.evaluate(evaluate_dataset)

100/100 [==============================] - 33s 324ms/step - loss: 0.3004 - acc: 0.9019 - recall_1: 0.6103 - precision_1: 0.8713 - iou: 0.4502 - dice_coef: 0.5821


[0.300388365983963,
 0.9019368886947632,
 0.6102603077888489,
 0.871268630027771,
 0.45023319125175476,
 0.5820791721343994]

In [ ]:
for k in range(20):
  #print(k)
  y = read_mask("/content/drive/MyDrive/DIPLOMSKI/code/results/2021_02_07/"+ str(k) + "_GT_2021_02_07.png")
  y_pred = read_mask("/content/drive/MyDrive/DIPLOMSKI/code/results/2021_02_07/"+ str(k) + "_PRED_2021_02_07.png")
  lenIntersection=0

  for i in range(y.shape[0]):
      for j in range(y.shape[1]):
          if ( np.array_equal(y[i][j],y_pred[i][j]) ):
              lenIntersection+=1
  
  lenimg=y.shape[0]*y.shape[1]
  value = (2. * lenIntersection  / (2*lenimg))
  print(k, ": " ,value)

0 :  0.9696396993810787
1 :  0.6830293987621574
2 :  0.8234720380194518
3 :  0.6736958443854996
4 :  0.9749198717948718
5 :  0.9675480769230769
6 :  0.9609913793103448
7 :  0.9906388152077807
8 :  0.9575348143236074
9 :  0.8255305039787798
10 :  0.9490135941644562
11 :  0.9080680813439435
12 :  0.7995634394341291
13 :  0.9298767683465959
14 :  0.9698524535809019
15 :  0.9554072723253758
16 :  0.9353227232537578
17 :  0.9450956012378426
18 :  0.947026967285588
19 :  0.8733007294429708
